In [1]:
!pip install sentence-transformers pandas pinecone-client

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
!pip install neuralprophet

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytorch_lightning-1.9.5-py3-none-any.whl.metadata (23 kB)
   ---------------------------------------- 0.0/145.4 kB ? eta -:--:--
   ---------- ---------------------------- 41.0/145.4 kB 960.0 kB/s eta 0:00:01
   ---------------------------------------- 145.4/145.4 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.3 MB 5.7 MB/s eta 0:00:01
   ------------------------ --------------- 0.8/1.3 MB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/829.5 kB ? eta -:--:--
   --------------------------------------- 829.5/829.5 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.5.10
    Uninstalling pytorch-lightning-1.5.10:
      Successfully uninsta

# Pinecone and Dataset

In [5]:
import os
import math
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import logging
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from neuralprophet import NeuralProphet

C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [6]:
# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
# Initialize Pinecone
pc = Pinecone(api_key="09bb980b-6cef-48c3-9aa5-63f3cbc9885e")
index = pc.Index("financial-data-index")

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['C:\\Users\\Pham Ty\\AppData\\Roaming\\Python\\Python312\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone


In [8]:
# Initialize SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
logger.info("SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
WARNING - (py.warnings._showwarnmsg) - C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(

  warnings.warn(

INFO:__main__:SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.


In [9]:
# Folder containing the CSV files
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'

# Folder to save the processed CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'

In [10]:
# Định nghĩa danh sách các mã chứng khoán
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
    'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
    'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
    'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
    'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
    'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
    'CMG', 'BCS', 'UAA']

In [11]:
def fetch_and_save_stock_data(symbol, start_date='2019-01-01', end_date='2024-01-01'):
    """
    Fetch stock data from Yahoo Finance, save to CSV, and upsert data into Pinecone.
    """
    try:
        # Fetch data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date)

        # Save data to CSV
        csv_file_path = os.path.join(folder_path, f'{symbol}.csv')
        stock_data.to_csv(csv_file_path)
        logger.info(f"Data for {symbol} saved to {csv_file_path}")

        # Load CSV data
        df = pd.read_csv(csv_file_path)

        # Select necessary columns and drop NaN values
        required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        if not all(col in df.columns for col in required_columns):
            logger.error(f"{csv_file_path} is missing one or more required columns: {required_columns}")
            return

        df = df[required_columns].dropna()

        # Convert each row into a single string of concatenated data
        text_data = df.apply(
            lambda row: f"Date: {row['Date']}, Open: {row['Open']}, High: {row['High']}, Low: {row['Low']}, Close: {row['Close']}, Adj Close: {row['Adj Close']}, Volume: {row['Volume']}",
            axis=1
        )

        # Generate embeddings for the stock data
        embeddings = embedding_model.encode(text_data.tolist(), convert_to_numpy=True)

        # Prepare data for Pinecone upsert
        vectors = [(f"{symbol}_{i}", embeddings[i].tolist(), {'symbol': symbol}) for i in range(len(embeddings))]

        # Upsert data into Pinecone index
        try:
            response = index.upsert(vectors)
            logger.info(f"Successfully upserted {len(vectors)} vectors for {symbol}.")
            logger.info(f"Pinecone response: {response}")
        except Exception as e:
            logger.error(f"Error upserting data into Pinecone for {symbol}: {e}")

        # Save the processed DataFrame to a new CSV file in the output folder
        processed_file_path = os.path.join(output_folder, f'processed_{symbol}.csv')
        df.to_csv(processed_file_path, index=False)
        logger.info(f"Processed data saved to {processed_file_path}")

    except Exception as e:
        logger.error(f"Error processing data for {symbol}: {e}")

In [15]:
# Loop through all symbols and process each
for symbol in symbols:
    fetch_and_save_stock_data(symbol)

[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for NVDA saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\NVDA.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for NVDA: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 07:55:42 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1321', 'x-pinecone-request-id': '6984336462633659435', 'x-envoy-upstream-service-time': '14', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_NVDA.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for INTC saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\INTC.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for INTC: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 07:55:57 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '2084', 'x-pinecone-request-id': '4306332076737310382', 'x-envoy-upstream-service-time': '15', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_INTC.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for PLTR saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\PLTR.csv


...

ERROR:__main__:Error upserting data into Pinecone for CMG: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 08:35:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '4881', 'x-pinecone-request-id': '8393789949870308419', 'x-envoy-upstream-service-time': '14', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_CMG.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for BCS saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\BCS.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for BCS: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 08:35:23 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '4941', 'x-pinecone-request-id': '2942809384439832523', 'x-envoy-upstream-service-time': '52', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_BCS.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for UAA saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\UAA.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for UAA: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 08:35:42 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '5508', 'x-pinecone-request-id': '7209779187538689079', 'x-envoy-upstream-service-time': '15', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_UAA.csv


In [12]:
csv_files = ['NVDA.csv', 'INTC.csv', 'PLTR.csv', 'TSLA.csv', 'AAPL.csv', 'BBD.csv', 'T.csv', 'SOFI.csv',
             'WBD.csv', 'SNAP.csv', 'NIO.csv', 'BTG.csv', 'F.csv', 'AAL.csv', 'NOK.csv', 'BAC.csv',
             'CCL.csv', 'ORCL.csv', 'AMD.csv', 'PFE.csv', 'KGC.csv', 'MARA.csv', 'SLB.csv', 'NU.csv',
             'MPW.csv', 'MU.csv', 'LCID.csv', 'NCLH.csv', 'RIG.csv', 'AMZN.csv', 'ABEV.csv', 'U.csv',
             'LUMN.csv', 'AGNC.csv', 'VZ.csv', 'WBA.csv', 'WFC.csv', 'RIVN.csv', 'UPST.csv', 'GRAB.csv',
             'CSCO.csv', 'VALE.csv', 'AVGO.csv', 'PBR.csv', 'GOOGL.csv', 'SMMT.csv', 'GOLD.csv',
             'CMG.csv', 'BCS.csv', 'UAA.csv']

In [13]:
# Kiểm tra các tệp có trong thư mục
csv_files_in_folder = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Lấy các mã từ tên các file CSV
csv_symbols_in_folder = [f.split('.')[0] for f in csv_files_in_folder]

# Kiểm tra các mã cổ phiếu nào thiếu
missing_symbols = [symbol for symbol in symbols if symbol not in csv_symbols_in_folder]

# In ra kết quả
if missing_symbols:
    print(f"Các mã cổ phiếu bị thiếu: {missing_symbols}")
else:
    print("Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.")

Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.


# Neural Prophet Model

In [14]:
# Tải dữ liệu và chuẩn bị dữ liệu cho NeuralProphet
def prepare_neuralprophet_data(df):
    # Đổi tên cột cho đúng định dạng của NeuralProphet
    df = df.rename(columns={'Date': 'ds', 'Close': 'y'})
    return df

# Huấn luyện mô hình NeuralProphet
def build_neuralprophet_model():
    model = NeuralProphet(daily_seasonality=True)  # Kích hoạt tính mùa vụ hàng ngày
    return model

In [15]:
# Đọc dữ liệu từ CSV cho một mã cổ phiếu
def load_stock_data(symbol):
    file_path = os.path.join(folder_path, f'{symbol}.csv')  # Thêm '.csv' vào cuối tên tệp
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df = df[['Date', 'Close']]  # Chỉ giữ lại cột Date và Close
    df = df.rename(columns={'Date': 'ds', 'Close': 'y'})  # Đổi tên cột cho phù hợp với NeuralProphet
    return df

In [16]:
# Initialize a list to store forecast results
forecast_results = []

# Loop through each symbol and perform forecasting
for symbol in symbols:
    print(f"Processing {symbol}...")

    # Load data
    df = load_stock_data(symbol)

    # Ensure data is sorted by date
    df = df.sort_values('ds').reset_index(drop=True)

    # Determine the split index for 80:20 train:test
    split_index = int(len(df) * 0.8)
    train_df = df.iloc[:split_index].copy()
    test_df = df.iloc[split_index:].copy()

    # Initialize the NeuralProphet model with daily seasonality
    model = NeuralProphet(daily_seasonality=True)

    # Fit the model on the training data
    model.fit(train_df, freq='D')

    # Create a dataframe to hold future dates (test period)
    future = model.make_future_dataframe(train_df, periods=len(test_df), n_historic_predictions=True)

    # Predict on the test period
    forecast = model.predict(future)

    # Extract the predicted values for the test period
    y_pred = forecast['yhat1'].iloc[-len(test_df):].values

    # Extract the actual values for the test period
    y_test = test_df['y'].values

    # Calculate performance metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # Extract training prices
    train_prices = train_df['y'].tolist()

    # Append the results to the forecast_results list
    forecast_results.append({
        'Symbol': symbol,
        'RMSE': rmse,
        'MSE': mse,
        'MAPE': mape,
        'Train_Prices': train_prices,               # List of training prices
        'Test_Actual_Prices': y_test.tolist(),      # Actual prices in test set
        'Test_Predicted_Prices': y_pred.tolist(),   # Predicted prices in test set
        'Forecast_2024': forecast['yhat1'].iloc[-len(test_df):].tolist()  # Forecasted prices for the test period
    })

    # Optionally, save the forecast DataFrame to the output folder for each symbol
    forecast_output_path = os.path.join(output_folder, f'{symbol}_forecast.csv')
    forecast.to_csv(forecast_output_path, index=False)
    print(f"Forecast for {symbol} saved to {forecast_output_path}")

# Create a summary DataFrame from the forecast_results
summary_df = pd.DataFrame(forecast_results)

# Save the summary to a CSV file
summary_csv_path = os.path.join(output_folder, 'forecast_summary_neural_prophet.csv')
summary_df.to_csv(summary_csv_path, index=False)
print(f"Forecast summary saved to {summary_csv_path}")

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency B corresponds to 96.322% of the data.
INFO:NP.df_utils:Major frequency B corresponds to 96.322% of the data.
WARNING - (py.warnings._showwarnmsg) - C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_date

Processing NVDA...


WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (63) is too small than the required number                     for the learning rate finder (233). The results might not be optimal.


Finding best initial lr:   0%|          | 0/233 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\lightning_fabric\utilities\cloud_io.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

...

Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency B corresponds to 96.322% of the data.
INFO:NP.df_utils:Major frequency B corresponds to 96.322% of the data.
WARNING - (py.warnings._showwarnmsg) - C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNI

Predicting: 63it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


Forecast for UAA saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\UAA_forecast.csv
Forecast summary saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\forecast_summary_neural_prophet.csv


In [17]:
# Tạo bảng tổng hợp kết quả dự báo
summary_neuralprophet = pd.DataFrame(forecast_results)
summary_neuralprophet

,Symbol,RMSE,MSE,MAPE,Train_Prices,Test_Actual_Prices,Test_Predicted_Prices,Forecast_2024
0,NVDA,42.549031,1810.420053,0.574339,"[0.4860000014305115, 0.4769999980926513, 0.472...","[12.928999900817873, 13.197999954223633, 13.38...","[24.50663185119629, 24.485090255737305, 12.580...","[24.50663185119629, 24.485090255737305, 12.580..."
1,INTC,13.928182,193.994246,0.347821,"[35.16999816894531, 34.81999969482422, 34.7099...","[28.84000015258789, 29.239999771118164, 29.440...","[39.95179748535156, 39.87004852294922, 34.3172...","[39.95179748535156, 39.87004852294922, 34.3172..."
2,PLTR,10.337212,106.857961,0.323875,"[9.5, 9.460000038146973, 9.199999809265137, 9....","[20.049999237060547, 20.270000457763672, 18.39...","[21.084890365600586, 20.982128143310547, 36.89...","[21.084890365600586, 20.982128143310547, 36.89..."
3,TSLA,211.786071,44853.339693,1.012893,"[17.58799934387207, 17.288000106811523, 16.668...","[303.75, 303.3500061035156, 309.07000732421875...","[336.3050537109375, 336.11798095703125, 410.92...","[336.3050537109375, 336.11798095703125, 410.92..."
4,AAPL,75.846313,5752.663133,0.314337,"[25.44750022888184, 25.239999771118164, 25.264...","[152.3699951171875, 150.6999969482422, 154.479...","[173.95913696289062, 173.9166259765625, 27.495...","[173.95913696289062, 173.9166259765625, 27.495..."
5,BBD,1.293799,1.673917,0.418882,"[11.30854034423828, 10.99862289428711, 10.7988...","[3.660000085830689, 3.5799999237060547, 3.7699...","[3.672856092453003, 3.7111921310424805, 3.9891...","[3.672856092453003, 3.7111921310424805, 3.9891..."
6,T,6.254668,39.120874,0.236425,"[26.555891036987305, 26.79002952575684, 26.812...","[16.760000228881836, 16.739999771118164, 16.76...","[18.441768646240234, 18.512121200561523, 22.58...","[18.441768646240234, 18.512121200561523, 22.58..."
7,SOFI,12.119666,146.886311,0.902432,"[12.199999809265137, 12.149999618530272, 12.11...","[9.9399995803833, 9.3100004196167, 9.640000343...","[7.323999404907227, 7.2468061447143555, 7.0948...","[7.323999404907227, 7.2468061447143555, 7.0948..."
8,WBD,17.981032,323.317516,1.353372,"[39.970001220703125, 40.5, 39.34999847412109, ...","[13.170000076293944, 12.739999771118164, 13.15...","[20.361330032348633, 20.405385971069336, 23.62...","[20.361330032348633, 20.405385971069336, 23.62..."
9,SNAP,59.594224,3551.471592,4.687909,"[24.479999542236328, 27.09000015258789, 23.770...","[10.34000015258789, 11.09000015258789, 10.6999...","[-7.679438591003418, -7.746350288391113, -8.16...","[-7.679438591003418, -7.746350288391113, -8.16..."


In [1]:
import os
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import logging
from datetime import timedelta
import json
from dash import dash_table

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Paths to data
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'  # Folder with stock CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'  # Output folder

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
           'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
           'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
           'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
           'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
           'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
           'CMG', 'BCS', 'UAA']

# Load forecast_summary_neural_prophet.csv
forecast_summary_file = os.path.join(output_folder, 'forecast_summary_neural_prophet.csv')
if os.path.exists(forecast_summary_file):
    forecast_summary_df = pd.read_csv(forecast_summary_file)
    # Parse string lists into actual lists using json.loads
    list_columns = ['Train_Prices', 'Test_Actual_Prices', 'Test_Predicted_Prices', 'Forecast_2024']
    for col in list_columns:
        if col in forecast_summary_df.columns:
            try:
                forecast_summary_df[col] = forecast_summary_df[col].apply(
                    lambda x: json.loads(x) if pd.notnull(x) else []
                )
                logging.info(f"Parsed column '{col}' successfully.")
            except json.JSONDecodeError as e:
                logging.error(f"Error decoding JSON in column '{col}': {e}")
                forecast_summary_df[col] = [[] for _ in range(len(forecast_summary_df))]
    logging.info(f"Loaded '{forecast_summary_file}' successfully.")
else:
    forecast_summary_df = pd.DataFrame()
    logging.warning(f"Could not find '{forecast_summary_file}'. Evaluation metrics and forecasts will not be displayed.")

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Define app layout
app.layout = html.Div([
    html.H1('Stock Price Predictions Dashboard - Neural Prophet'),
    html.Div([
        html.Label('Select Stock Symbol'),
        dcc.Dropdown(
            id='stock-dropdown',
            options=[{'label': s, 'value': s} for s in symbols],
            value='NVDA'  # Default stock
        )
    ], style={'width': '25%', 'display': 'inline-block'}),
    html.Div([
        html.Label('Select Date Range'),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed='2014-09-18',  # Adjusted min date for 10-year period
            max_date_allowed='2025-09-18',  # Extended by 1 year for forecasting
            start_date='2014-09-18',        # Default start date
            end_date='2025-09-18'           # Extended by 1 year for forecasting
        )
    ], style={'display': 'inline-block', 'marginLeft': '50px'}),
    html.Div([
        html.Label('Show 1-Year Forecast'),
        dcc.Checklist(
            id='forecast-checkbox',
            options=[{'label': 'Include 1-Year Forecast', 'value': 'show_forecast'}],
            value=[],
            inline=True
        )
    ], style={'marginTop': '20px'}),
    dcc.Graph(id='price-graph'),
    html.Div(id='metrics-output', style={'marginTop': '20px'}),
    html.Div([
        html.Button("Download Predictions", id="download-button"),
        dcc.Download(id="download-predictions")
    ], style={'marginTop': '20px'}),
    html.Div(id='download-info', style={'marginTop': '10px'})
])

# Define callback to update graph and evaluation metrics
@app.callback(
    [Output('price-graph', 'figure'),
     Output('metrics-output', 'children')],
    [Input('stock-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date'),
     Input('forecast-checkbox', 'value')]
)
def update_graph(selected_stock, start_date, end_date, forecast_option):
    logging.info(f"Updating graph for stock: {selected_stock}")
    # Convert start_date and end_date to datetime
    try:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        logging.info(f"Selected date range: {start_date} to {end_date}")
    except Exception as e:
        logging.error(f"Error parsing dates: {e}")
        return {}, "Error parsing dates."

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        logging.error(f"No forecast data found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"No forecast data found for {selected_stock}."

    # Parse 'Test_Actual_Prices', 'Test_Predicted_Prices', 'Forecast_2024', 'Train_Prices'
    actual_prices = forecast_row.iloc[0].get('Test_Actual_Prices', [])
    predicted_prices = forecast_row.iloc[0].get('Test_Predicted_Prices', [])
    future_prices = forecast_row.iloc[0].get('Forecast_2024', [])
    train_prices = forecast_row.iloc[0].get('Train_Prices', [])

    logging.info(f"Lengths - Train: {len(train_prices)}, Test Actual: {len(actual_prices)}, Test Predicted: {len(predicted_prices)}, Forecast: {len(future_prices)}")

    # Check future_prices data
    if not future_prices:
        logging.warning(f"No future price predictions available for {selected_stock}.")
    else:
        logging.info(f"Loaded {len(future_prices)} future price predictions for {selected_stock}.")

    # Load the stock data to get dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    if not os.path.exists(stock_data_file):
        logging.error(f"Stock data file not found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Stock data file not found for {selected_stock}."

    df_stock = pd.read_csv(stock_data_file)
    if 'Date' not in df_stock.columns or 'Close' not in df_stock.columns:
        logging.error(f"Incorrect data format in stock data file for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Incorrect data format in stock data file for {selected_stock}."

    # Convert 'Date' to datetime
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    dates = df_stock['Date']
    data_length = len(df_stock)

    # Assuming the data was split into 80% train and 20% test
    train_size = int(len(dates) * 0.8)
    test_size = len(dates) - train_size

    logging.info(f"Total data length: {len(dates)}, Train size: {train_size}, Test size: {test_size}")

    # Ensure that the length of 'Test_Actual_Prices' matches 'test_size'
    if len(actual_prices) != test_size or len(predicted_prices) != test_size:
        logging.error(f"Length mismatch between actual/predicted prices and test set for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Length mismatch in data for {selected_stock}."

    # Dates for train and test sets
    dates_train = dates.iloc[:train_size].reset_index(drop=True)
    dates_test = dates.iloc[train_size:].reset_index(drop=True)

    # Check if lengths match
    if len(train_prices) != len(dates_train):
        logging.warning(f"Length of train_prices ({len(train_prices)}) does not match dates_train ({len(dates_train)}). Adjusting.")
        min_len = min(len(train_prices), len(dates_train))
        train_prices = train_prices[:min_len]
        dates_train = dates_train[:min_len]

    if len(actual_prices) != len(dates_test) or len(predicted_prices) != len(dates_test):
        logging.warning(f"Length of test prices does not match dates_test for {selected_stock}. Adjusting.")
        min_len_test = min(len(actual_prices), len(predicted_prices), len(dates_test))
        actual_prices = actual_prices[:min_len_test]
        predicted_prices = predicted_prices[:min_len_test]
        dates_test = dates_test[:min_len_test]

    logging.info(f"After adjustment - Train: {len(train_prices)}, Test Actual: {len(actual_prices)}, Test Predicted: {len(predicted_prices)}")

    # Create DataFrame for train set
    df_train = pd.DataFrame({
        'Date': dates_train,
        'Train_Price': train_prices
    })

    # Create DataFrame for test set
    df_test = pd.DataFrame({
        'Date': dates_test,
        'Actual_Price': actual_prices,
        'Predicted_Price': predicted_prices
    })

    # Filter df_train and df_test based on 'start_date' and 'end_date'
    df_train_filtered = df_train[(df_train['Date'] >= start_date) & (df_train['Date'] <= end_date)]
    df_test_filtered = df_test[(df_test['Date'] >= start_date) & (df_test['Date'] <= end_date)]

    logging.info(f"Filtered data - Train: {len(df_train_filtered)}, Test: {len(df_test_filtered)}")

    # Initialize data list for plotting
    data = []

    # Add Train Prices if available in the selected date range
    if not df_train_filtered.empty:
        data.append(
            go.Scatter(
                x=df_train_filtered['Date'],
                y=df_train_filtered['Train_Price'],
                mode='lines',
                name='Train Price',
                line=dict(color='red')
            )
        )
        logging.info(f"Added Train Price trace with {len(df_train_filtered)} points.")

    # Add Actual Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Actual_Price'],
                mode='lines',
                name='Actual Test Price',
                line=dict(color='green')
            )
        )
        logging.info(f"Added Actual Test Price trace with {len(df_test_filtered)} points.")

    # Add Predicted Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Predicted_Price'],
                mode='lines',
                name='Predicted Test Price',
                line=dict(color='blue')
            )
        )
        logging.info(f"Added Predicted Test Price trace with {len(df_test_filtered)} points.")

    # Add future price predictions if selected
    if 'show_forecast' in forecast_option and future_prices:
        # Generate future dates
        last_date = dates.iloc[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices), freq='D')

        if len(future_prices) != len(future_dates):
            logging.error(f"Length mismatch between future prices ({len(future_prices)}) and future dates ({len(future_dates)}) for {selected_stock}.")
        else:
            df_future = pd.DataFrame({
                'Date': future_dates,
                'Future_Price': future_prices
            })
            # Filter future predictions based on date range
            df_future_filtered = df_future[(df_future['Date'] >= start_date) & (df_future['Date'] <= end_date)]
            logging.info(f"Future predictions after filtering: {len(df_future_filtered)}")
            if not df_future_filtered.empty:
                data.append(
                    go.Scatter(
                        x=df_future_filtered['Date'],
                        y=df_future_filtered['Future_Price'],
                        mode='lines',
                        name='1-Year Forecast',
                        line=dict(dash='dash', color='orange')
                    )
                )
                logging.info(f"Added 1-Year Forecast trace with {len(df_future_filtered)} points.")

    # Create figure
    figure = {
        'data': data,
        'layout': go.Layout(
            title=f'{selected_stock} Price Prediction - Neural Prophet',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'},
            hovermode='closest'
        )
    }

    # Get RMSE, MSE, and MAPE
    rmse = forecast_row.iloc[0].get('RMSE', None)
    mse = forecast_row.iloc[0].get('MSE', None)
    mape = forecast_row.iloc[0].get('MAPE', None)

    logging.info(f"Metrics - RMSE: {rmse}, MSE: {mse}, MAPE: {mape}")

    if rmse is not None and mse is not None and mape is not None:
        # Create metrics DataFrame
        metrics_df = pd.DataFrame({
            'Metric': ['RMSE', 'MSE', 'MAPE'],
            'Value': [f"{rmse:.4f}", f"{mse:.4f}", f"{mape:.2%}"]
        })

        metrics_output = html.Div([
            html.H4('Evaluation Metrics'),
            dash_table.DataTable(
                columns=[{"name": i, "id": i} for i in metrics_df.columns],
                data=metrics_df.to_dict('records'),
                style_cell={'textAlign': 'left'},
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                },
                style_table={'width': '50%'}
            )
        ])
        logging.info("Added metrics table.")
    else:
        metrics_output = html.Div([
            html.H4('Evaluation Metrics'),
            html.P("Metrics not available.")
        ])
        logging.warning("Metrics not available.")

    return figure, metrics_output

# Define callback for downloading predictions
@app.callback(
    Output("download-predictions", "data"),
    [Input("download-button", "n_clicks")],
    [dash.dependencies.State('stock-dropdown', 'value')],
    prevent_initial_call=True,
)
def download_predictions(n_clicks, selected_stock):
    logging.info(f"Download button clicked for stock: {selected_stock}")
    if n_clicks is None:
        logging.info("No clicks detected.")
        return dash.no_update

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        logging.error(f"No forecast data found for {selected_stock}.")
        return dash.no_update

    # Extract data
    actual_prices = forecast_row.iloc[0].get('Test_Actual_Prices', [])
    predicted_prices = forecast_row.iloc[0].get('Test_Predicted_Prices', [])
    future_prices = forecast_row.iloc[0].get('Forecast_2024', [])
    train_prices = forecast_row.iloc[0].get('Train_Prices', [])

    logging.info(f"Lengths for download - Train: {len(train_prices)}, Test Actual: {len(actual_prices)}, Test Predicted: {len(predicted_prices)}, Forecast: {len(future_prices)}")

    # Load stock dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    if not os.path.exists(stock_data_file):
        logging.error(f"Stock data file not found for {selected_stock}.")
        return dash.no_update

    df_stock = pd.read_csv(stock_data_file)
    if 'Date' not in df_stock.columns or 'Close' not in df_stock.columns:
        logging.error(f"Incorrect data format in stock data file for {selected_stock}.")
        return dash.no_update

    # Convert 'Date' to datetime
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    dates = df_stock['Date']

    # Assuming the data was split into 80% train and 20% test
    train_size = int(len(dates) * 0.8)
    test_size = len(dates) - train_size

    # Ensure that the length of 'Test_Actual_Prices' matches 'test_size'
    if len(actual_prices) != test_size or len(predicted_prices) != test_size:
        logging.error(f"Length mismatch between actual/predicted prices and test set for {selected_stock}.")
        return dash.no_update

    # Prepare dates for test set
    dates_test = dates.iloc[train_size:].reset_index(drop=True)

    # Prepare DataFrame for download
    download_df = pd.DataFrame({
        'Date': dates_test,
        'Actual_Price': actual_prices,
        'Predicted_Price': predicted_prices
    })

    logging.info(f"Prepared test set data for download with {len(download_df)} rows.")

    # Add future predictions if any
    if future_prices:
        last_date = dates.iloc[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices), freq='D')
        download_future_df = pd.DataFrame({
            'Date': future_dates,
            'Future_Price_Prediction': future_prices
        })
        download_df = pd.concat([download_df, download_future_df], ignore_index=True)
        logging.info(f"Added future predictions to download data with {len(download_future_df)} rows.")

    # Convert DataFrame to CSV
    logging.info(f"Sending download data for {selected_stock}.")
    return dcc.send_data_frame(download_df.to_csv, f'{selected_stock}_neural_prophet_predictions.csv', index=False)

# Run Dash app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

2024-10-02 21:29:00,504 INFO:Parsed column 'Train_Prices' successfully.
2024-10-02 21:29:00,514 INFO:Parsed column 'Test_Actual_Prices' successfully.
2024-10-02 21:29:00,523 INFO:Parsed column 'Test_Predicted_Prices' successfully.
2024-10-02 21:29:00,533 INFO:Parsed column 'Forecast_2024' successfully.
2024-10-02 21:29:00,534 INFO:Loaded 'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\forecast_summary_neural_prophet.csv' successfully.
